In [50]:
import json
train_data = []
with open('train.json', 'r') as file:
    for line in file:
        train_data.append(json.loads(line.strip()))

In [51]:
# json.loads(train_data[0]['Functions'][0])
# Fix this error: JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
# Convert single quotes to double quotes before parsing JSON
# I want to format this as a json object and not a string

fixed_json = train_data[0]['Functions'][0].replace("'", '"')

j = json.dumps(json.loads(fixed_json),indent=1)


In [52]:
j

'{\n "name": "Torch",\n "api_name": "torch.linspace",\n "description": "Create a one-dimensional tensor with evenly spaced values",\n "parameters": {\n  "start": {\n   "type": "float",\n   "description": "The starting value for the set of points"\n  },\n  "end": {\n   "type": "float",\n   "description": "The ending value for the set of points"\n  },\n  "steps": {\n   "type": "int",\n   "description": "The number of evenly spaced values to generate"\n  },\n  "out": {\n   "type": "Tensor",\n   "description": "Optional output tensor"\n  },\n  "dtype": {\n   "type": "torch.dtype",\n   "description": "Optional data type for the computation"\n  },\n  "layout": {\n   "type": "torch.layout",\n   "description": "Optional layout of the returned tensor"\n  },\n  "device": {\n   "type": "torch.device",\n   "description": "Optional device for the returned tensor"\n  },\n  "requires_grad": {\n   "type": "bool",\n   "description": "Optional flag to enable gradient tracking"\n  }\n }\n}'

In [1]:
from typing import Literal
import json
import yaml
import ast
from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import get_chat_template
from datasets import Dataset
import functools
from trl import SFTTrainer
from transformers import TrainingArguments
import tqdm
from unsloth import is_bfloat16_supported
def json_to_yaml(data): 
    curr_func_yaml = ""
    json_func = ast.literal_eval(data)
    for func in json_func:
        curr_func_yaml+=yaml.dump(ast.literal_eval(func)) + "\n\n"
    return curr_func_yaml
def _process_bfcl_train_data(train_data):
    for td in tqdm.tqdm(train_data):
        output = td['Output']
        # print(td['Functions'][0])
        # fixed_json = td['Functions'][0].replace("'", '"')
        # fixed_json = td['Functions'][0].replace("'", '"').replace('True', 'true').replace('False', 'false')
        # First parse with ast to handle Python literal syntax
        parsed_dict = ast.literal_eval(td['Functions'][0])
        # Then convert to proper JSON string
        fixed_json = json.dumps(parsed_dict)
        td['json_function'] = json.dumps(json.loads(fixed_json),indent=1)
        td['Functions'] = str(td['Functions'])
        td['yaml_function'] = json_to_yaml(td['Functions'])
        del td['Functions']
        
        if isinstance(output,str):
            pass
        elif isinstance(output,list):
            output = output[0]
        td['Output'] = output
    return train_data

train_data = []
with open('train.json', 'r') as file:
    for line in file:
        train_data.append(json.loads(line.strip()))
with open('test.json', 'r') as file:
    test_data = json.load(file)
train_data = _process_bfcl_train_data(train_data)

dtype = torch.bfloat16 
load_in_4bit = False 

/home/recoverx/.cache/pypoetry/virtualenvs/finetuning-KL8mpKMW-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/recoverx/.cache/pypoetry/virtualenvs/finetuning-KL8mpKMW-py3.10/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


100%|██████████| 12125/12125 [01:13<00:00, 164.11it/s]


In [2]:
train_data[100]

{'Instruction': 'I want to replace null elements in two arrays, arr1 and arr2, with corresponding fill values fill1 and fill2.\n',
 'Output': 'pyarrow.fill_null(arr1,fill1)',
 'json_function': '{\n "name": "pyarrow",\n "api_name": "fill_null",\n "description": "Replace null elements in an array",\n "parameters": {\n  "required": [\n   {\n    "name": "values",\n    "description": "Array, ChunkedArray, or Scalar-like object. Each null element is replaced with the corresponding value from fill_value."\n   },\n   {\n    "name": "fill_value",\n    "description": "Array, ChunkedArray, or Scalar-like object. If not the same type as data, will attempt to cast."\n   }\n  ]\n }\n}',
 'yaml_function': "api_name: fill_null\ndescription: Replace null elements in an array\nname: pyarrow\nparameters:\n  required:\n  - description: Array, ChunkedArray, or Scalar-like object. Each null element is\n      replaced with the corresponding value from fill_value.\n    name: values\n  - description: Array, Ch

## REMAINING WORK
1. Evaluation for BFCL and xLAM dataset
2. finetuning scripts and integration with weights and biases

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Llama-3.2-1B-Instruct",
        # max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        trust_remote_code=True
    )

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.325 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [4]:
train_ds = Dataset.from_list(train_data)
def _create_messages(user_prompt:str,functions:str, output:str):
    messages = [
            {
                "role": "system",
                "content": "You are an expert in composing functions. You are given a question and a set of possible functions."
                            " Based on the question, you will need to make one or more function/tool calls to achieve the purpose."
                            " If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n",
            },
            {"role": "user", "content": f"#### Question: {user_prompt}Here is a list of functions in JSON format that you can invoke:\n{functions}. Should you decide to return the function call(s), NO other text MUST be included.\n#### Response:"},
            {"role":"assistant","content":output}
        ]
    return messages

def _get_tokenized_ds(examples,tokenizer, json_or_yaml: Literal["json","yaml"]):
    user_prompts = examples['Instruction']
    functions = examples['json_function'] if json_or_yaml == "json" else examples["yaml_function"]
    out = examples['Output']
    prompts = []
    for up,fn,op in zip(user_prompts,functions,out):
        prompts.append(tokenizer.apply_chat_template(
            _create_messages(up,fn, op),
            tokenize=False
        ))

    return {"prompt":prompts,}

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
train_ds = train_ds.map(functools.partial(_get_tokenized_ds,tokenizer=tokenizer,json_or_yaml="json"),batched=True)

Map: 100%|██████████| 12125/12125 [00:00<00:00, 22044.74 examples/s]


In [6]:
print(train_ds[100]['prompt'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose. If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.
<|eot_id|><|start_header_id|>user<|end_header_id|>

#### Question: I want to replace null elements in two arrays, arr1 and arr2, with corresponding fill values fill1 and fill2.
Here is a list of functions in JSON format that you can invoke:
{
 "name": "pyarrow",
 "api_name": "fill_null",
 "description": "Replace null elements in an array",
 "parameters": {
  "required": [
   {
    "name": "values",
    "description": "Array, ChunkedArray, or Scalar-like object. Each

In [7]:
msg =  _create_messages(user_prompt=train_data[0]['Instruction'],functions=train_data[0]['json_function'], output=train_data[0]['Output'])

In [8]:
msg

[{'role': 'system',
  'content': 'You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose. If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n'},
 {'role': 'user',
  'content': '#### Question: I want to create a one-dimensional tensor with evenly spaced values from 0 to 1 using the torch.linspace API.\nHere is a list of functions in JSON format that you can invoke:\n{\n "name": "Torch",\n "api_name": "torch.linspace",\n "description": "Create a one-dimensional tensor with evenly spaced values",\n "parameters": {\n  "start": {\n   "type": "float",\n   "description": "The starting value for the set of points"\n  },\n  "end": {\n   "type": "float",\n   "description": "The ending value for the set

In [9]:
print(train_ds[0]['prompt'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose. If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.
<|eot_id|><|start_header_id|>user<|end_header_id|>

#### Question: I want to create a one-dimensional tensor with evenly spaced values from 0 to 1 using the torch.linspace API.
Here is a list of functions in JSON format that you can invoke:
{
 "name": "Torch",
 "api_name": "torch.linspace",
 "description": "Create a one-dimensional tensor with evenly spaced values",
 "parameters": {
  "start": {
   "type": "float",
   "description": "The starting value for the set o

In [10]:
print(tokenizer.apply_chat_template(
 msg,
 tokenize=False,
#  add_generation_prompt=True   
))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose. If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.
<|eot_id|><|start_header_id|>user<|end_header_id|>

#### Question: I want to create a one-dimensional tensor with evenly spaced values from 0 to 1 using the torch.linspace API.
Here is a list of functions in JSON format that you can invoke:
{
 "name": "Torch",
 "api_name": "torch.linspace",
 "description": "Create a one-dimensional tensor with evenly spaced values",
 "parameters": {
  "start": {
   "type": "float",
   "description": "The starting value for the set o

In [11]:
model = FastLanguageModel.get_peft_model(
        model,
        r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj",],
        lora_alpha = 64,
        lora_dropout = 0, # Supports any, but = 0 is optimized
        bias = "none",    # Supports any, but = "none" is optimized
        # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
        use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
        random_state = 3407,
        use_rslora = False,  # We support rank stabilized LoRA
        loftq_config = None, # And LoftQ
    )

Unsloth 2024.9.post4 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [12]:
from unsloth.chat_templates import train_on_responses_only
from transformers import DataCollatorForSeq2Seq

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    dataset_text_field = "prompt",
    max_seq_length = 3072,
    dataset_num_proc = 8,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 1,
        # warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        save_safetensors=False
    ),
)
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>system<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)
trainer_stats = trainer.train()

Map: 100%|██████████| 12125/12125 [00:03<00:00, 3987.91 examples/s]
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 12,125 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 1
\        /    Total batch size = 32 | Total steps = 379
 "-____-"     Number of trainable parameters = 22,544,384


Step,Training Loss
1,1.264800
2,0.785200
3,0.826100
4,0.589300
5,0.331200
6,0.542900
7,0.473500
8,0.521800
9,0.444800
10,0.458700


In [15]:
# from unsloth.chat_templates import train_on_responses_only

# trainer = train_on_responses_only(
#     trainer,
#     instruction_part = "<|start_header_id|>system<|end_header_id|>\n\n",
#     response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
# )

Map:   0%|          | 0/12125 [00:00<?, ? examples/s]

Map: 100%|██████████| 12125/12125 [00:05<00:00, 2293.12 examples/s]


In [13]:
print(tokenizer.decode(trainer.train_dataset[100]["input_ids"]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose. If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.
<|eot_id|><|start_header_id|>user<|end_header_id|>

#### Question: I want to replace null elements in two arrays, arr1 and arr2, with corresponding fill values fill1 and fill2.
Here is a list of functions in JSON format that you can invoke:
{
 "name": "pyarrow",
 "api_name": "fill_null",
 "description": "Replace null elements in an array",
 "parameters": {
  "required": [
   {
    "name": "values",
    "description": "Array, ChunkedArray, or Scalar-like object. Each

In [14]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                               \n\npyarrow.Time32()<|eot_id|>'

In [15]:
import json

with open('test.json', 'r') as file:
    test_data = json.load(file)

In [16]:
idx = 100
test_data[idx]['function']

{'name': 'Waterfall Finder',
 'api_call': 'waterfall.find_highest',
 'description': 'Find the highest waterfall in the world.',
 'parameters': {'type': 'object', 'properties': {}, 'required': []}}

In [29]:
test_data[idx]['model_answer']

'waterfall.find_highest()'

In [17]:
import yaml

yaml_func = yaml.dump(test_data[idx]['function'])

In [18]:
print(yaml_func)

api_call: waterfall.find_highest
description: Find the highest waterfall in the world.
name: Waterfall Finder
parameters:
  properties: {}
  required: []
  type: object



In [19]:
# import finetune
prompt_messages = _create_messages(
    test_data[idx]['question'],
    functions=json.dumps(test_data[idx]['function'],indent=1),
    # functions=yaml_func,
    output=""
)

In [20]:
prompt_messages

[{'role': 'system',
  'content': 'You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose. If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n'},
 {'role': 'user',
  'content': '#### Question: Find the highest waterfall in the world.Here is a list of functions in JSON format that you can invoke:\n{\n "name": "Waterfall Finder",\n "api_call": "waterfall.find_highest",\n "description": "Find the highest waterfall in the world.",\n "parameters": {\n  "type": "object",\n  "properties": {},\n  "required": []\n }\n}. Should you decide to return the function call(s), NO other text MUST be included.\n#### Response:'},
 {'role': 'assistant', 'content': ''}]

In [21]:
prompt = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)

In [22]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose. If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.
<|eot_id|><|start_header_id|>user<|end_header_id|>

#### Question: Find the highest waterfall in the world.Here is a list of functions in JSON format that you can invoke:
{
 "name": "Waterfall Finder",
 "api_call": "waterfall.find_highest",
 "description": "Find the highest waterfall in the world.",
 "parameters": {
  "type": "object",
  "properties": {},
  "required": []
 }
}. Should you decide to return the function call(s), NO other text MUST be included.
#### Re

In [23]:
model.dtype, model.device

(torch.bfloat16, device(type='cuda', index=0))

In [24]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose. If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.
<|eot_id|><|start_header_id|>user<|end_header_id|>

#### Question: Find the highest waterfall in the world.Here is a list of functions in JSON format that you can invoke:
{
 "name": "Waterfall Finder",
 "api_call": "waterfall.find_highest",
 "description": "Find the highest waterfall in the world.",
 "parameters": {
  "type": "object",
  "properties": {},
  "required": []
 }
}. Should you decide to return the function call(s), NO other text MUST be included.
#### Re

In [25]:
model.config.torch_dtype = torch.bfloat16

In [26]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [27]:
import unsloth
unsloth.FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt
], return_tensors = "pt").to("cuda:0")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True, do_sample=False)
out = tokenizer.batch_decode(outputs)

In [28]:
print(out[0])

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose. If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.
<|eot_id|><|start_header_id|>user<|end_header_id|>

#### Question: Find the highest waterfall in the world.Here is a list of functions in JSON format that you can invoke:
{
 "name": "Waterfall Finder",
 "api_call": "waterfall.find_highest",
 "description": "Find the highest waterfall in the world.",
 "parameters": {
  "type": "object",
  "properties": {},
  "required": []
 }
}. Should you decide to return the function call(s), NO other text MUST be 

In [46]:
from finetune import parse_python_function_call

# parse_python_function_call(out[0].)
python_output = out[0][out[0].rfind("<|end_header_id|>",1)+19:-10].strip()

In [47]:
python_output

'gcloud.alpha.ai.models().delete(MODEL="my-model", region="us-west1")'

In [37]:
import ast

def process_ast_node(node):
    if isinstance(node, (ast.Constant, ast.Constant, ast.Constant)):
        return node.value
    elif isinstance(node, ast.List):
        return [process_ast_node(elt) for elt in node.elts]
    else:
        return ast.unparse(node)

def parse_python_function_call(call_str):
    tree = ast.parse(call_str)
    expr = tree.body[0].value

    def extract_function_name(node):
        if isinstance(node, ast.Name):
            return node.id
        elif isinstance(node, ast.Attribute):
            return f"{extract_function_name(node.value)}.{node.attr}"
        else:
            return ast.unparse(node)
    # return expr
    function_name = extract_function_name(expr.func)

    parameters = {}
    noNameParam = []

    # Process positional arguments
    for arg in expr.args:
        noNameParam.append(process_ast_node(arg))

    # Process keyword arguments
    for kw in expr.keywords:
        parameters[kw.arg] = process_ast_node(kw.value)

    if noNameParam:
        parameters["None"] = noNameParam
        
    function_dict = {"name": function_name, "arguments": parameters}
    return function_dict

# Test the function
# call_str = "gcloud.active-directory.domains.trusts.update(DOMAIN='my-other-domain.com', target_dns_ip_addresses=['10.177.0.3'], target_domain_name='my-target-domain.com')"
result = parse_python_function_call(python_output)
print(result)

{'name': 'requests.get', 'arguments': {'headers': "{'X-RapidAPI-Key': 'SIGN-UP-FOR-KEY', 'X-RapidAPI-Host': 'minecraft-user-data.p.rapidapi.com'}", 'params': "{'name': 'Alex'}", 'None': ['https://minecraft-user-data.p.rapidapi.com/']}}


In [40]:
parse_python_function_call(test_data[11]['model_answer'])

{'name': 'news.get_headlines', 'arguments': {'source': 'CNN'}}

In [41]:
def evaluation(model_answer,gt_answer):
    model_answer = parse_python_function_call(model_answer)
    gt_answer = parse_python_function_call(gt_answer)

    if model_answer['name'] != gt_answer['name']:
        return 0
    for k,v in model_answer['arguments'].items():
        if k not in gt_answer['arguments'] or gt_answer['arguments'][k] != v:
            return 0
    return 1
evaluation(python_output,test_data[10]['model_answer'])

0